# BGT zadanie 5
## Autors
- Paweł Benkowski
- Stanisław Lutkiewicz"
### Task
Downloads Nietzsche and freud books count group of words in it(tiny,small,medium,big).  
Match language by analyzing character frequency  

In [1]:
import sys
!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install nltk

In [2]:
from pyspark import SparkContext, SparkFiles
from pathlib import Path
from bs4 import BeautifulSoup
from collections import Counter
from operator import add
import os
import requests
import PyPDF2
import re
import nltk
import timeit
nltk.download("averaged_perceptron_tagger")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
def get_books_urls(url):
    urlsWithName = []
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    for a in soup.find_all("a", href=True):
        arr = a["href"].split(".")
        if arr[-1] == "pdf":
            urlsWithName.append(("http:" +a["href"],a.string))
    return urlsWithName

In [4]:
def get_books_and_create_folder(urls, target):
    if not os.path.isdir(target):
        os.makedirs(target)
    for url in urls:
        file = Path(target,url[1] + ".pdf")
        if not file.exists():
            response = requests.get(url[0])
            with open(Path(target, url[1] + ".pdf"), "wb+") as f:
                f.write(response.content)
        else:
            print("File already exists")


In [5]:
def save_text_from_pdf(name):
    file = Path(name + ".txt")
    if not file.exists():
        fileReader = PyPDF2.PdfFileReader(name)
        with open(Path(name + ".txt"), "a+") as f:
            for page in fileReader.pages:
                f.write(page.extractText())

In [6]:
def is_noun(word : str) -> bool:
    if "s" == word or "i" == word:
        return False
    partOfSpeech = nltk.pos_tag([word])
    if "NN" == partOfSpeech[0][1]:
        return True
    return False

In [7]:
def compose_text_files(textFiles):
    print(textFiles)
    if len(textFiles) == 1 :
        return textFiles[0].name
    with open(textFiles[0],'a') as toAppend:
        for txt in textFiles[1::]:
            with open(txt) as fromAppend:
                toAppend.write(fromAppend.read())
            os.remove(txt)
    print(textFiles)    
    return textFiles[0].name

In [8]:
sc = SparkContext("local", "first app")

In [9]:
folderNamesWithUrls = [('freud',"https://holybooks.com/sigmund-freud-the-complete-works/"),\
    ('nitsche',"https://holybooks.com/the-complete-works-of-friedrich-nietzsche-in-english-as-pdf/")]
files = []
for fnwu in folderNamesWithUrls:
    urls = get_books_urls(fnwu[1])
    get_books_and_create_folder(urls, fnwu[0])
    pdfFiles = list(Path(fnwu[0]).glob('*.pdf'))
    for pdf in pdfFiles:
        save_text_from_pdf(os.path.join(fnwu[0], pdf.name))
    textFileName = compose_text_files(list(Path(fnwu[0]).glob('*.txt')))
    files.append(sc.textFile(os.path.join(fnwu[0], textFileName)).cache())

File already exists
[PosixPath('freud/Sigmund Freud – The Complete Works.pdf.txt')]
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
File already exists
[PosixPath('nitsche/The complete works of Friedrich Nietzsche VOL XVI.pdf.txt'), PosixPath('nitsche/The complete works of Friedrich Nietzsche VOL XI.pdf.txt'), PosixPath('nitsche/The complete works of Friedrich Nietzsche VOL XVII.pdf.txt'), PosixPath('nitsche/The complete works of Friedrich Nietzsche VOL V.pdf.txt'), PosixPath('nitsche/The complete works of Friedrich Nietzsche VOL X.pdf.txt'), PosixPath('nitsche/The complete works of Friedrich Nietzsche VOL XIV.pdf.txt'), PosixPath('nitsche/The complete works of Friedrich Nietzsche VOL XVIII.pdf.txt'

In [10]:
def count_letters_and_print_ratio_and_language(lettersRdd):
    lettersForCounting = lowerCaseLetters.map(lambda x: (x,1))
    countedLetters =  lettersForCounting.reduceByKey(add).collect()
    kLetters = Counter(dict(countedLetters))
    mostCommonLetters = kLetters.most_common(3)
    allLetters = lowerCaseLetters.count()
    print(f"top letters {mostCommonLetters}")
    print(f"total letters {allLetters}")
    mostCommonLettersRatio = {}
    for mcl in mostCommonLetters:
        mostCommonLettersRatio[mcl[0]] = (mcl[1]/allLetters)*100
        print(f"{mcl[0]} : {mostCommonLettersRatio[mcl[0]]}%")
    print(match_language(mostCommonLettersRatio))


def split_ratio(ratioString):
    ratioDict = {}
    for l in ratioString.split(","):
        ls = l.split(":")
        ratioDict[ls[0].lower()]=float(ls[1])
    return ratioDict


def match_language(topLetters):
    englishRatio = "A:8.55,B:1.60,C:3.16,D:3.87,E:12.10,F:2.18,G:2.09,H:4.96,I:7.33,J:0.22,K:0.81,L:4.21,M:2.53,N:7.17,O:7.47,P:2.07,Q:0.10,R:6.33,S:6.73,T:8.94,U:2.68,V:1.06,W:1.83,X:0.19,Y:1.72,Z:0.11"
    polishRatio = "a:8.91,i:8.21,o:7.75,e:7.66,z:5.64,n:5.52,r:4.69,w:4.65,s:4.32,t:3.98,c:3.96,y:3.76,k:3.51,d:3.25,p:3.13,m:2.80,u:2.50,j:2.28,l:2.10,ł:1.82,b:1.47,g:1.42,ę:1.11,h:1.08,ą:0.99,ó:0.85,ż:0.83,ś:0.66,ć:0.40,f:0.30,ń:0.20,q:0.14,ź:0.06,v:0.04,x:0.02"
    englishRatioDict = split_ratio(englishRatio)
    polishRatioDict = split_ratio(polishRatio)
    winners = {}
    for key,value in topLetters.items():
        if(value-englishRatioDict[key] > value-polishRatioDict[key]):
            winners[key] = "polish"
        else:
            winners[key] = "english"
    polishWinCounter=0
    for key,value in winners.items():
        if value == "polish":
            polishWinCounter+=1
    if(polishWinCounter>2):
        return "polish"
    else:
        return "english"

def create_groups_and_print_count(words):
    kTiny = words.filter(lambda x: len(x) < 2 ).count()
    kSmall = words.filter(lambda x: 1 < len(x) < 5 ).count()
    kMedium = words.filter(lambda x: 4 < len(x) < 10 ).count()
    kBig = words.filter(lambda x: len(x) > 9 ).count()
    print(f"tiny {kTiny}")
    print(f"small {kSmall}")
    print(f"medium {kMedium}")
    print(f"big {kBig}")
    


In [11]:
start = timeit.default_timer()
# pfiles = sc.parallelize(files).foreach(do_work)
for file in files: # paralize -> foreach
    startIn = timeit.default_timer()
    print(file.name)
    lowerCaseText = file.map(lambda x: x.lower())
    lowerCaseLetters = lowerCaseText.flatMap(lambda x: re.findall(r"[a-zA-Z]", x))
    count_letters_and_print_ratio_and_language(lowerCaseLetters)

    words = lowerCaseText.flatMap(lambda x : re.findall(r"\b[^\d\W]+\b", x))
    create_groups_and_print_count(words)

    wordsForCounting = words.map(lambda x: (x,1))
    countedNouns = wordsForCounting.reduceByKey(add).filter(lambda x: is_noun(x[0])).collect()

    kNouns = Counter(dict(countedNouns))
    kWords = Counter(words.collect())
    
    print(f"total word {words.count()}")
    print(f"most common {kWords.most_common(10)}")
    print(f"top nouns {kNouns.most_common(3)}")
    print(f"time spend {timeit.default_timer() - startIn}")
    print(30*"----")
print(f"total time spend {timeit.default_timer()-start}")

<bound method RDD.name of freud/Sigmund Freud – The Complete Works.pdf.txt MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0>
top letters [('e', 1204065), ('t', 931463), ('i', 759718)]
total letters 9572627
e : 12.578208677722428%
t : 9.730484641258872%
i : 7.936358535645439%
english
tiny 81428
small 1152818
medium 693893
big 140767
total word 2068906
most common [('the', 142371), ('of', 94924), ('to', 60988), ('in', 50950), ('a', 48808), ('and', 46208), ('that', 33805), ('is', 30431), ('it', 27788), ('which', 22649)]
top nouns [('dream', 6115), ('analysis', 3584), ('time', 3456)]
time spend 54.0358791059698
------------------------------------------------------------------------------------------------------------------------
<bound method RDD.name of nitsche/The complete works of Friedrich Nietzsche VOL XVI.pdf.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0>
top letters [('e', 3632535), ('t', 2790736), ('i', 2285251)]
total letters 29295089
e 

In [14]:
sc.addFile("hdfs://namenode:9000/covid_19_india.csv")


# 'hdfs:///user/bekce/myfile.json'

covid_19_india.csv  freud  nitsche  nltk_data  work
/home/jovyan


/bin/bash: hadoop: command not found
